In [200]:
import numpy as np
import theano
import theano.tensor as T

In [201]:
import fbTaskReader as taskReader
reload(taskReader)

# Get the input matrices for the facebook tasks
taskFilePaths = ["/Users/SaahilM/Documents/Princeton/Academics/Thesis/Data/tasks_1-20_v1-2/en/qa1_single-supporting-fact"]
task1 = taskReader.VectorizeTask(taskFilePaths[0])
qaPairs = task1.getTextQuestionPairs()

In [202]:
# theano.config.compute_test_value = 'warn'

# Number of training examples
N = task1.getNumTrainingExamples()
# Number of words in our dictionary
V = task1.getNumWords()
# Dimension to encode our information. They use 20 for independent training
d = 20

print(N,V,d)

# Input X is the collection of sentences, Vx(text_length) matrix
# Input q is our query, a vector of size Vx1
# The actual result is y, a vector of size Vx1
X = T.lmatrix('X')
q = T.lrow('q')
y = T.lvector('y')

X.tag.test_value = np.zeros((2, V), dtype=np.int64)
q.tag.test_value = np.zeros((2,V), dtype=np.int64)

1000 19 20


In [203]:
# Create a weight matrix of given size. 
# The matrix is initialized randomly with Gaussian distribution 
# with mean=0 and \sigma=0.1
def initializeWeightMatrix(in_size, out_size):
    return theano.shared(np.random.randn(in_size, out_size))

# Create a bias vector of all zeros of given size
def initializeBiasVector(size):
    return theano.shared(np.zeros(size,))

In [210]:
# Initialize all our parameters, given our dimensions.
# A is the first matrix used to embed our input. It has size Vxd
# B is the matrix used to embed the query. It has size Vxd
# C is the next matrix used to embed our input. It has size Vxd
# W is the final matrix. Takes output O and produces result R. It has size dxV
# H is the linear mapping we apply to u_i in each layer, since we have set it
#      up like an RNN
def initializeParams(d, V):
    A = initializeWeightMatrix(V, d)
    B = initializeWeightMatrix(V, d)
    C = initializeWeightMatrix(V, d)
    W = initializeWeightMatrix(d, V)
    H = initializeWeightMatrix(d, d) #H is just a vector
    
#     A = theano.shared(initializeWeightMatrix(d, V))
#     B = theano.shared(initializeWeightMatrix(d, V))
#     C = theano.shared(initializeWeightMatrix(d, V))
#     W = theano.shared(initializeWeightMatrix(V, d))
    return A, B, C, W, H

A, B, C, W, H = initializeParams(d, V)
weightMatrices = [A, B, C, W, H]
# for weightMatrix in weightMatrices:
#     print weightMatrix.eval()
# print(theano.shared(np.zeros((1,V), dtype=np.int64)).dot(B).shape.eval())

(False, False)


In [205]:
# Define the computational step
# Given input matrix X, query q, and weight matrices, we perform a computational step,
# also known as a "hop". Let M be the number of sentences
def hopComputation(prior_result, X, A, B, C, W, H):
    # m_i = Ax_i
    mem_matrix = X.dot(A) #dimension (MxV)x(Vxd) = Mxd
    
    u = prior_result
    
    # p_i = softmax(u^T m_i)
    probs = T.nnet.softmax(mem_matrix.dot(u.T)) #dimension (Mxd)x(dx1) = Mx1
    
    # C_i = Cx_i
    c_embedded = X.dot(C) #dimension (MxV)x(Vxd) = Mxd
    # output = sum of c_matrix * probs
    o = (probs * c_embedded).sum(axis = 0) #dimension = 1xd
    o = o.dimshuffle('x', 0)
    o = T.unbroadcast(o, 0)
    
    layer_result = o + u.dot(H) #dimension 1xd
#     layer_result = T.unbroadcast(layer_result, 1)
    
    print(prior_result.ndim)
    print(u.ndim)
    print(mem_matrix.ndim)
    print(probs.ndim)
    print(c_embedded.ndim)
    print(o.ndim)
    print(layer_result.ndim)
    
    print(prior_result.broadcastable)
    print(u.broadcastable)
    print(mem_matrix.broadcastable)
    print(probs.broadcastable)
    print(c_embedded.broadcastable)
    print(o.broadcastable)
    print(layer_result.broadcastable)
    
    return layer_result
    #For each hop computation, we will return the layer_result vector. We will return a 
    # final result multiplied by vector W at the very end

In [211]:
# Here we do the scan step 3 times (for our 3 hops)
initial_value = q.dot(B)#.dimshuffle('x',0)
print(initial_value.ndim)
initial_value = T.unbroadcast(initial_value, 0)
print (B.ndim)
print(B.broadcastable)
print(initial_value.broadcastable)
hops_result, updates = theano.scan(fn=hopComputation,
                                   n_steps=3,
                                   outputs_info=initial_value,
                                   non_sequences=[X] + weightMatrices,
                                   )
real_hops_result = hops_result[-1]
print("breaking")
print(hops_result.ndim)
print(hops_result.broadcastable)
print(real_hops_result.ndim)
print(real_hops_result.broadcastable)
pre_softmax = real_hops_result.dot(W)
y_hat = T.nnet.softmax(pre_softmax)
print(y_hat.ndim)
print(y_hat.broadcastable)
# y_hat = T.unbroadcast(y_hat, 1)
# y_hat = T.unbroadcast(y_hat, 0)
# q.fill(theano.shared(np.random.randn(1,4)))
# print q.eval()

2
2
(False, False)
(False, False)
2
2
2
2
2
2
2
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
breaking
3
(False, False, False)
2
(False, False)
2
(False, False)


In [207]:
#Learning rate
epsilon = 0.04
loss = T.nnet.categorical_crossentropy(y_hat, y).mean()
print(y.ndim)
print(y.broadcastable)
print(loss.ndim)
print(loss.broadcastable)

1
(False,)
0
()


In [208]:
from __future__ import print_function

def inspect_inputs(i, node, fn):
    print(i, node, "input(s) value(s):", fn.inputs, end='')

def inspect_outputs(i, node, fn):
    print(" output(s) value(s):", fn.outputs)
    
def detect_nan(i, node, fn):
    for output in fn.outputs:
        if (not isinstance(output[0], np.random.RandomState) and
            np.isnan(output[0]).any()):
            print('*** NaN detected ***')
            theano.printing.debugprint(node)
            print('Inputs : %s' % [input[0] for input in fn.inputs])
            print('Outputs: %s' % [output[0] for output in fn.outputs])
            break

# This function trains our neural net, using stochastic gradient descent.
def train_MemNN(loss, X, q, y):
    update_weights = []
    for weightMatrix in weightMatrices:
        update = T.grad(loss, weightMatrix)
        update_weights.append((weightMatrix, weightMatrix - update * epsilon))
    train_MemNN_func = theano.function(inputs=[X,q,y], outputs=loss, updates=update_weights, 
                        mode=theano.compile.MonitorMode(
#                             pre_func=inspect_inputs,
                            post_func=detect_nan))
    return train_MemNN_func

train_MemNN_func = train_MemNN(loss, X, q, y)

In [209]:
def train_model(train_data, epochs=1):
    train_errors = []
    for i in xrange(epochs):
        error = 0
#         print(train_data[0])
        for textQuestionPair in train_data:
            train_X = textQuestionPair["text"]
            print(train_X.shape)
            train_q = textQuestionPair["question"]
            print(train_q.shape)
            train_y = textQuestionPair["answer"]
            print(train_y.shape)
#             print train_y.dtype
#             train_X = np.array(textQuestionPair["text"])
#             train_q = np.array(textQuestionPair["question"])
#             train_y = np.array(textQuestionPair["answer"])
            cur_loss = train_MemNN_func(train_X, train_q, train_y)
            error += cur_loss
        train_errors.append(error)
    return train_errors

train_errors = train_model(qaPairs)
print(train_errors)

(2, 19)
(1, 19)
(1,)


ValueError: Input dimension mis-match. (input[0].shape[1] = 1, input[1].shape[1] = 20)
Apply node that caused the error: Elemwise{Mul}[(0, 0)](Softmax.0, <TensorType(float64, matrix)>)
Inputs types: [TensorType(float64, matrix), TensorType(float64, matrix)]

HINT: Use another linker then the c linker to have the inputs shapes and strides printed.
HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.
Apply node that caused the error: forall_inplace,cpu,scan_fn}(TensorConstant{3}, IncSubtensor{InplaceSet;:int64:}.0, <TensorType(float64, matrix)>, dot.0, dot.0)
Inputs types: [TensorType(int8, scalar), TensorType(float64, 3D), TensorType(float64, matrix), Tenso